In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sqlite3
import xml.etree.ElementTree as ET

from dateutil.parser import parse

In [3]:
#Establishing Connection
conn = sqlite3.connect('vs.db')
cur = conn.cursor()

In [4]:
folderpath = './varfind/'
dat_filename = 'varfind.dat'
img_filename = 'varfind.fts'
xml_filename = 'varfind.xml'
indata = np.loadtxt(folderpath + dat_filename, delimiter=" ")
e = ET.parse(folderpath + xml_filename).getroot()

image_file = fits.open(folderpath+img_filename)
image_data = image_file[0].data
image_header = image_file[0].header

JD = image_header["JD"]
exposure = image_header["EXPTIME"]
filter_type = image_header["FILTER"]
date = parse(image_header["DATE-OBS"]).date()
location = '('+image_header["RA"]+', '+image_header["DEC"]+')'

#Creating Arrays for Data
JD = indata[:, 0]
mag_JD = indata[:, 1::2]
mag_star = np.transpose(mag_JD)
magerror_JD = indata[:, ::2]
magerror_JD = magerror_JD[:, 1:]

image_file.close()

In [5]:
#Saving FITS Image
fig = plt.figure(figsize=(12,12))

plt.imshow(image_data, cmap='gray')
plt.colorbar()

fig.suptitle('NGC 2180 Star Field | Exposure: ' + str(exposure) + '| Filter: ' + filter_type, fontsize=20)

fig.savefig('./lightcurves/'+'field' + str(date) + '.png')

plt.close()

In [14]:
#Inserting Stack of Images in VSDB
cur.execute('SELECT count(StackID) FROM StackTable')
res = cur.fetchone()
#Determining the StackId for the given stack
stackID = res[0] + 1

string = (stackID, date, filter_type, exposure, location)
cur.execute('INSERT INTO StackTable(StackID, Date, Filter, Exptime, Location) VALUES (?,?,?,?,?)', string)

conn.commit()

1


In [15]:
#Assigning Ref coordinates to stars and storing in DB
for star in e.iter('s'):
    loc = '('+star.get('x') + ', ' + star.get('y')+')'
    string = (stackID, star.get('id'), loc)
    cur.execute('INSERT INTO StarTable(StackID, StarID, S_Location) VALUES (?,?,?)', string)

conn.commit()

In [22]:
for n in range(0, len(mag_star)):
    mag = mag_star[n].tolist()
    nJD = JD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)
        else:
            string = (n+1, mag[(len(mag_star[n]) - 1) - idx], nJD[(len(mag_star[n]) - 1) - idx])
            cur.execute('INSERT INTO S_LightCurve(StarID, JD, Star_Mag) VALUES (?,?,?)', string)

conn.commit()

In [22]:
#TRANSLATE XY COORDS TO RA DEC
#FIND PERIODS OF LCs

In [21]:
#Pulling Lightcurve from DB
cur.execute('SELECT Star_Mag, JD FROM S_LightCurve WHERE StarID = 1;')
res = np.array(cur.fetchall())
print(res[:,0])

[ 2457710.8735822  2457710.8712674  2457710.8689988  2457710.8667535
  2457710.8645313  2457710.8623206  2457710.8593808  2457710.8570891
  2457710.8548322  2457710.8525289  2457710.8503067  2457710.8480498
  2457710.8457928  2457710.843478   2457710.8411748  2457710.8389988
  2457710.8367188  2457710.8345313  2457710.8322512  2457710.8299595
  2457710.8277025  2457710.8253877  2457710.8231308  2457710.8208391
  2457710.8185937  2457710.8163137  2457710.8140451  2457710.8118113
  2457710.8095775  2457710.8072859  2457710.8050174  2457710.802853
  2457710.8006192  2457710.7983391  2457710.7960243  2457710.7937326
  2457710.7915104  2457710.7891609  2457710.7868576  2457710.7846123
  2457710.7823206  2457710.7800752  2457710.777772   2457710.775434
  2457710.7731655  2457710.770897   2457710.7686516  2457710.7664063
  2457710.764184   2457710.7618808  2457710.759647   2457710.7574132
  2457710.7551331  2457710.752853   2457710.7505498  2457710.7483623
  2457710.7461285  2457710.7438715  

In [23]:
#If a comp star does not have a data point, do you remove all data points for all comp stars or negate that star from the average for that point
def defCompStars(stars):
    comp_mag_list = [mag_star[s] for s in stars]
    comp_JD = JD.tolist()
    
    comp_mag = np.zeros_like(comp_mag_list[0])
    '''for s in stars:
        comp_mag = np.add(comp_mag, mag_star[s])
        
    comp_mag /= len(stars)
    #Is this step necessary
    comp_mag = np.ndarray.tolist(comp_mag)
    
    #Popping Comp Star Values where a comp is null
    for idx, val in enumerate(reversed(comp_mag)):
        del_el = False
        index = (len(mag_star[0]) - 1) - idx
        for n in stars:
            del_el = del_el or (mag_star[n][index] >= 99)
        if del_el:
            print(index)
            comp_mag.pop(index)
            comp_JD.pop(index)
    '''
    return comp_mag, comp_JD

In [24]:
c_mag, c_JD = defCompStars([1, 2, 500])
print(len(c_mag))

100


In [25]:
#Plotting the nth star's light curve
def plotLightCurve(n):
    
    mag = mag_star[n].tolist()
    nJD = JD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()

In [29]:
plotLightCurve(516)

# Links
https://d17h27t6h515a5.cloudfront.net/topher/2016/September/57ed880e_sql-sqlite-commands-cheat-sheet/sql-sqlite-commands-cheat-sheet.pdf
https://s3.amazonaws.com/michael-kennedy/downloads/DM/python_cheat_sheet.pdf
